In [ ]:
%pip install llama-index
%pip install llama-index-embeddings-openai
%pip install llama-index-llms-openai
%pip install llama-index-readers-file
%pip install docx2txt
%pip install llama-index-vector-stores-faiss
%pip install faiss-cpu

In [2]:
from google.colab import drive
drive.mount('/content/drive')
input_dir = '/content/drive/MyDrive/colab_input_data/docs/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

EMBED_DIMENSION = 512

In [4]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.faiss import FaissVectorStore
import faiss

In [5]:
# initialize the LLM
llm = OpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY)

# initialize the embedding
embed_model = OpenAIEmbedding(model="text-embedding-3-small", api_key=OPENAI_API_KEY, dimensions=EMBED_DIMENSION)

In [6]:
from llama_index.core import Settings

# global settings
Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 200
Settings.chunk_overlap = 50

In [7]:
# load documents
node_parser = SimpleDirectoryReader(input_dir=input_dir,required_exts=[".docx"])
documents = node_parser.load_data()

In [8]:

# Create FaisVectorStore to store embeddings
faiss_index = faiss.IndexFlatL2(EMBED_DIMENSION)
vector_store = FaissVectorStore(faiss_index=faiss_index)

In [9]:
from llama_index.core.schema import BaseNode, TransformComponent
from llama_index.core.text_splitter import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline

class TextCleaner(TransformComponent):
    """
    Transformation to be used within the ingestion pipeline.
    Cleans clutters from texts.
    """
    def __call__(self, nodes, **kwargs) -> list[BaseNode]:

        for node in nodes:
          if 'text_resource' in node:
              node.text_resource = node.text_resource.replace('\t', ' ') # Replace tabs with spaces
              node.text_resource = node.text_resource.replace(' \n', ' ') # Replace paragraph seperator with spacaes

        return nodes

In [10]:
text_splitter = SentenceSplitter(chunk_size=200, chunk_overlap=50)

# Create a pipeline with defined document transformations and vectorstore
pipeline = IngestionPipeline(
    transformations=[
        TextCleaner(),
        text_splitter,
    ],
    vector_store=vector_store,
)

In [11]:
# Run pipeline and get generated nodes from the process
nodes = pipeline.run(documents=documents)

In [12]:
vector_store_index = VectorStoreIndex(nodes)
query_engine = vector_store_index.as_retriever(similarity_top_k=3)

In [13]:
# generating query response
response = query_engine.retrieve("where Paul learned ?")

In [14]:
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.chat_engine import CondensePlusContextChatEngine

# creating chat memory buffer
memory = ChatMemoryBuffer.from_defaults(token_limit=4500)

# creating chat engine
chat_engine = CondensePlusContextChatEngine.from_defaults(
   query_engine,
   memory=memory,
   llm=llm
)

# generating chat response
response = chat_engine.chat(
   "where Paul learned ?"
)
print(str(response))


Paul Yudkin studied at Bar-Ilan University, where he earned both his M.Sc. in Financial Mathematics with a grade of 88 and his B.Sc. in Computer Science & Applied Mathematics with a grade of 83.


In [15]:
# generating chat response
response = chat_engine.chat(
   "what you can tell about htis place where he leared?"
)
print(str(response))

Bar-Ilan University is located in Ramat Gan, Israel, and is known for its strong emphasis on both academic excellence and Jewish values. It was established in 1955 and has grown to become one of the largest universities in Israel, offering a wide range of undergraduate and graduate programs across various fields, including humanities, social sciences, natural sciences, and engineering.

The university is also recognized for its research contributions, particularly in areas like computer science, mathematics, and the social sciences. It has a vibrant campus life, with numerous student organizations, cultural events, and opportunities for community engagement. Additionally, Bar-Ilan University promotes a unique blend of secular and religious studies, making it a distinctive institution in the Israeli higher education landscape.


In [16]:
# generating chat response
response = chat_engine.chat(
   "what are the universities at that country ? please give short description"
)
print(str(response))

Israel has several prominent universities, each known for its unique strengths and areas of focus. Here are some of the key institutions:

1. **Hebrew University of Jerusalem**: Founded in 1918, it is Israel's oldest university and is renowned for its research and academic excellence. It offers a wide range of programs and is particularly strong in the humanities, social sciences, and natural sciences.

2. **Tel Aviv University**: Established in 1956, it is one of the largest universities in Israel. Known for its innovative research and diverse academic programs, it excels in fields such as business, law, and the arts.

3. **Technion - Israel Institute of Technology**: Located in Haifa, the Technion is Israel's premier engineering and technology university, founded in 1912. It is known for its cutting-edge research and contributions to science and technology.

4. **Bar-Ilan University**: As mentioned earlier, Bar-Ilan combines academic rigor with Jewish studies. It offers a variety of 